In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### KAIST Scene Text Database 전처리기:
1. 데이터 url:http://www.iapr-tc11.org/mediawiki/index.php/KAIST_Scene_Text_Database
1. 압축을 다 풀고 'kaist_dataset'아래에 이미지 데이터와 xml 파일을 두면
1. 각 xml 파일에서 글자하나하나 위치정보를 읽어 원본 이미지에서 글자를 별도의 이미지 파일로 저장하는 작업을 수행한다.
1. 결과물은 char_data 아래에 각 글자명으로 폴더를 만들고 그 아래에 원본파일명.글자명.idx.jpg로 저장
    - char_data/{character}/{img_filename}.{character}.{idx}.jpg
    - character가 .(dot)인 경우에는 {character}값을 dot으로 대체하여 처리
    

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import xml.etree.ElementTree

from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

img_base_path = 'kaist_dataset'

def plot_img_inline(img):
    plt.figure()
    plt.imshow(np.asarray(img))
    plt.show()

char_count = dict()

def get_unique_img_name(target_path, index_name, char_name):
    idx = 0
    image_filename = '.'.join([index_name, char_name, str(idx), 'jpg'])
    full_path = join(target_path, image_filename)
    
    while path.exists(full_path):
        idx = idx + 1
        image_filename = '.'.join([index_name, char_name, str(idx), 'jpg'])
        full_path = join(target_path, image_filename)
    return full_path

def print_img(img, base_path, index_name, char_name):
    target_path = join(base_path, char_name)
    if not path.exists(target_path):
        makedirs(target_path)

    # 이미지 데이터들이 여러 폴더에 걸쳐있어서 같은 파일명과 같은 글자를 갖는 경우가 많음
    # 뒤에 idx값을 추가해 서로 다른 데이터가 다른 파일명을 갖도록 함.
    full_path = get_unique_img_name(target_path, index_name, char_name)
    img.save(full_path)
    
    if char_name not in char_count:
        char_count[char_name] = 1
    else:
        char_count[char_name] = char_count[char_name] + 1
    
def crop_character_images(base_path, index_name, output_path='char_data'):
    image_filename = '.'.join([index_name, 'jpg'])
    xml_filename = '.'.join([index_name, 'xml'])
    
    # xml 파일에 확장자를 빠트린 경우가 많음. xml 파일이 없는 경우에는 xml파일명을 확장자를 제외한 것으로 설정
    if path.exists(join(base_path, xml_filename)) is False:
        xml_filename = index_name
    im = Image.open(join(base_path, image_filename))
    
#     plot_img_inline(im)
    e = xml.etree.ElementTree.parse(join(base_path, xml_filename)).getroot()

    for image in e:
        for words in image:
            if words.tag == 'words':
                for word in words:
                    for char in word:
                        if len(char.attrib['char']) != 1:
                            continue
                            
                        # 글자가 .인경우 dot으로 대체
                        if char.attrib['char'] == '.':
                            char.attrib['char'] = 'dot'
                            
                        # box 영역으로 crop
                        box = (int(char.attrib['x']), int(char.attrib['y']), int(char.attrib['x']) + int(char.attrib['width']), int(char.attrib['y']) + int(char.attrib['height']))
                        region = im.crop(box)
#                         plot_img_inline(region)

                        # 파일로 저장
                        print_img(region, output_path, index_name, char.attrib['char'])

In [3]:
from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

img_base_path = 'kaist_dataset'

fail_filenames = []
def get_recursive_dirlist(base_path):
    # dirname을 얻어옴
    sub_dirnames = [f for f in listdir(base_path) if isdir(join(base_path, f))]
    for dirname in sub_dirnames:
        # 각 dirname별로  recursive 호출
        next_base_path = join(base_path, dirname)
        get_recursive_dirlist(next_base_path)
        
        # 현재 dir에 jpg파일이 있는지 확인
        # KAIST데이터셋은 xxx.jpg, xxx.xml, xxx.bmp 형식으로 있기 때문에, 
        # jpg파일 목록을 가져와서 이미지 데이터 목록을 확보
        data_filenames = [path.splitext(f)[0] for f in listdir(next_base_path) if isfile(join(next_base_path, f)) and f.lower().endswith('.jpg')]        
        for x in data_filenames:
            try:
                crop_character_images(next_base_path, x)
            except:
                # 데이터 형식이 잘못된 경우가 많아 exception이 발생한 경우는 그냥 버림. 대신 뭐가 문제였는지만 출력
                print('###EXCEPTION', next_base_path, x)
                fail_filenames.append((next_base_path, x))

# get_recursive_dirlist(img_base_path)

In [4]:
def read_charlist(base_path):
    char_data = {}
    sub_dirnames = [f for f in listdir(base_path) if isdir(join(base_path, f))]
    for dirname in sub_dirnames:
        # 각 dirname별로  recursive 호출
        next_base_path = join(base_path, dirname)
        
        # 현재 dir에 jpg파일이 있는지 확인
        # KAIST데이터셋은 xxx.jpg, xxx.xml, xxx.bmp 형식으로 있기 때문에, 
        # jpg파일 목록을 가져와서 이미지 데이터 목록을 확보
        data_filenames = [path.splitext(f)[0] for f in listdir(next_base_path) if isfile(join(next_base_path, f)) and f.lower().endswith('.jpg')]        
        
        char_data[dirname] = data_filenames
        
    return char_data
                
char_data = read_charlist('char_data')

dataset = {}
data_charset = {}
for idx, (char, data) in enumerate(char_data.items()):

    if len(data) < 300:
        continue
    print(idx, char, len(data))
    dataset[char] = data
    for x in data:
        data_charset[x] = char


12 0 819
13 1 630
14 2 828
15 3 462
16 4 798
17 5 522
18 6 497
19 7 524
20 8 967
21 9 360
26 A 843
28 C 502
31 E 1185
32 F 342
34 H 335
35 I 635
38 L 458
39 M 315
40 N 669
41 O 895
44 R 613
45 S 627
46 T 853
47 U 314
606 이 333
629 전 304


In [5]:
from functools import reduce
count = reduce(lambda x, y: x + y, [len(x) for k, x in dataset.items()])
print('count data', count, len(data_charset))

count data 15630 15630


In [6]:
data_charset

label_idx = {x:idx for idx, x in enumerate(sorted(list(set(data_charset.values()))))}

label_idx

data_idxcharset = {fname:label_idx[char] for fname, char in data_charset.items()}
data_idxcharset

{'005.0.0': 0,
 '005.0.1': 0,
 '006.0.0': 0,
 '013.0.0': 0,
 '017.0.0': 0,
 '023.0.0': 0,
 '027.0.0': 0,
 '032.0.0': 0,
 '035.0.0': 0,
 '042.0.0': 0,
 '047.0.0': 0,
 '047.0.1': 0,
 '047.0.2': 0,
 '047.0.3': 0,
 '056.0.0': 0,
 '056.0.1': 0,
 '080116-0058.0.0': 0,
 '080116-0059.0.0': 0,
 '080116-0059.0.1': 0,
 '080116-0065.0.0': 0,
 '080116-0065.0.1': 0,
 '080116-0065.0.2': 0,
 '080116-0084.0.0': 0,
 '080118-0007.0.0': 0,
 '080118-0007.0.1': 0,
 '080118-0007.0.2': 0,
 '080119-0003.0.0': 0,
 '080119-0009.0.0': 0,
 '080119-0009.0.1': 0,
 '080119-0018.0.0': 0,
 '080119-0018.0.1': 0,
 '080119-0018.0.2': 0,
 '080119-0018.0.3': 0,
 '080119-0018.0.4': 0,
 '080119-0018.0.5': 0,
 '080119-0018.0.6': 0,
 '080119-0018.0.7': 0,
 '080119-0030.0.0': 0,
 '080119-0031.0.0': 0,
 '080119-0031.0.1': 0,
 '090.0.0': 0,
 '091.0.0': 0,
 '10.0.0': 0,
 '105.0.0': 0,
 '11.0.0': 0,
 '11.0.1': 0,
 '121.0.0': 0,
 '123.0.0': 0,
 '123.0.1': 0,
 '123.0.2': 0,
 '125.0.0': 0,
 '126.0.0': 0,
 '126.0.1': 0,
 '128.0.0': 0,
 

In [7]:
label_idx

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'A': 10,
 'C': 11,
 'E': 12,
 'F': 13,
 'H': 14,
 'I': 15,
 'L': 16,
 'M': 17,
 'N': 18,
 'O': 19,
 'R': 20,
 'S': 21,
 'T': 22,
 'U': 23,
 '이': 24,
 '전': 25}

In [8]:
data_idxcharset

{'005.0.0': 0,
 '005.0.1': 0,
 '006.0.0': 0,
 '013.0.0': 0,
 '017.0.0': 0,
 '023.0.0': 0,
 '027.0.0': 0,
 '032.0.0': 0,
 '035.0.0': 0,
 '042.0.0': 0,
 '047.0.0': 0,
 '047.0.1': 0,
 '047.0.2': 0,
 '047.0.3': 0,
 '056.0.0': 0,
 '056.0.1': 0,
 '080116-0058.0.0': 0,
 '080116-0059.0.0': 0,
 '080116-0059.0.1': 0,
 '080116-0065.0.0': 0,
 '080116-0065.0.1': 0,
 '080116-0065.0.2': 0,
 '080116-0084.0.0': 0,
 '080118-0007.0.0': 0,
 '080118-0007.0.1': 0,
 '080118-0007.0.2': 0,
 '080119-0003.0.0': 0,
 '080119-0009.0.0': 0,
 '080119-0009.0.1': 0,
 '080119-0018.0.0': 0,
 '080119-0018.0.1': 0,
 '080119-0018.0.2': 0,
 '080119-0018.0.3': 0,
 '080119-0018.0.4': 0,
 '080119-0018.0.5': 0,
 '080119-0018.0.6': 0,
 '080119-0018.0.7': 0,
 '080119-0030.0.0': 0,
 '080119-0031.0.0': 0,
 '080119-0031.0.1': 0,
 '090.0.0': 0,
 '091.0.0': 0,
 '10.0.0': 0,
 '105.0.0': 0,
 '11.0.0': 0,
 '11.0.1': 0,
 '121.0.0': 0,
 '123.0.0': 0,
 '123.0.1': 0,
 '123.0.2': 0,
 '125.0.0': 0,
 '126.0.0': 0,
 '126.0.1': 0,
 '128.0.0': 0,
 

In [9]:
import tensorflow as tf


with tf.Graph().as_default():
    value = tf.read_file('char_data/1/077.1.1.jpg')
    img = tf.image.decode_jpeg(value)
    sess = tf.Session()
    print(sess.run(img))
    
    

[[[ 45  70 111]
  [ 45  70 111]
  [ 45  70 111]
  [ 45  70 111]
  [ 46  69 111]
  [ 46  69 111]
  [ 46  69 111]
  [ 46  69 111]
  [ 42  63 108]
  [ 51  72 117]
  [ 52  73 120]
  [ 45  66 113]
  [ 46  64 114]
  [ 52  70 120]]

 [[ 45  69 113]
  [ 45  69 113]
  [ 45  69 113]
  [ 45  69 113]
  [ 46  69 113]
  [ 46  69 113]
  [ 46  69 113]
  [ 46  69 113]
  [ 48  69 114]
  [ 47  68 115]
  [ 46  67 114]
  [ 46  67 114]
  [ 51  69 119]
  [ 54  72 122]]

 [[ 45  68 118]
  [ 45  68 118]
  [ 45  69 117]
  [ 45  69 117]
  [ 46  68 117]
  [ 46  68 117]
  [ 46  68 115]
  [ 46  68 115]
  [ 52  73 120]
  [ 46  66 116]
  [ 45  63 113]
  [ 48  66 116]
  [ 48  66 116]
  [ 42  60 110]]

 [[ 45  68 122]
  [ 45  68 122]
  [ 45  68 120]
  [ 45  68 120]
  [ 46  67 120]
  [ 46  68 118]
  [ 46  68 118]
  [ 46  68 117]
  [ 39  59 109]
  [ 43  63 113]
  [ 50  68 118]
  [ 54  72 122]
  [ 55  73 121]
  [ 55  73 121]]

 [[ 46  67 124]
  [ 46  67 124]
  [ 46  67 124]
  [ 46  67 124]
  [ 46  67 122]
  [ 46  67 122]


In [10]:
import numpy as np
label_arr = np.reshape(list(data_idxcharset.values()), (-1, 1))
label_arr

array([[ 0],
       [ 0],
       [ 0],
       ..., 
       [25],
       [25],
       [25]])

In [11]:
import tensorflow as tf
import cifar10


FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('eval_dir', '/tmp/kaist_eval',
                           """Directory where to write event logs.""")
tf.app.flags.DEFINE_string('eval_data', 'test',
                           """Either 'test' or 'train_eval'.""")
tf.app.flags.DEFINE_string('checkpoint_dir', '/tmp/kaist_train',
                           """Directory where to read model checkpoints.""")
tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5,
                            """How often to run the eval.""")
tf.app.flags.DEFINE_integer('num_examples', 1000,
                            """Number of examples to run.""")
tf.app.flags.DEFINE_boolean('run_once', False,
                            """Whether to run eval only once.""")


In [12]:
def get_inputs():
    # Create a queue that produces the filenames to read.
    filenames = ['char_data/' + label + '/' + char + '.jpg' for char, label in data_charset.items()]
    filename_queue = tf.train.string_input_producer(filenames)
    image_reader = tf.WholeFileReader()
    # Read a whole file from the queue, the first returned value in the tuple is the
    # filename which we are ignoring.
    _, image_file = image_reader.read(filename_queue)

    # Decode the image as a JPEG file, this will turn it into a Tensor which we can
    # then use in training.
    image_orig = tf.image.decode_jpeg(image_file)
    image = tf.image.resize_images(image_orig, [84, 84])
    image = tf.image.per_image_standardization(image)
    image.set_shape((84, 84, 3))

    tensor_label = tf.constant(list(data_idxcharset.values()), dtype=tf.int64)
    
    label_queue = tf.train.input_producer(tensor_label)
    i_reader = tf.IdentityReader()
    label, v = i_reader.read(label_queue)
    
    print('label_queue', label_queue)
    print('image_file', image_file)
    print('label', label, v)
    
    num_preprocess_threads = 1
    min_queue_examples = 256

    images, label_batch = tf.train.batch(
        [image, label_queue.dequeue()],
        batch_size=FLAGS.batch_size,
        num_threads=num_preprocess_threads,
        capacity=min_queue_examples + 3 * FLAGS.batch_size)

#     tf.summary.image('images', images)
#     tf.summary.histogram('label_batch', label_batch)
#     return images, tf.reshape(label_batch, [FLAGS.batch_size])
    return images, tf.reshape(label_batch, [FLAGS.batch_size])

get_inputs()

label_queue <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x118945198>
image_file Tensor("ReaderReadV2:1", shape=(), dtype=string)
label Tensor("ReaderReadV2_1:0", shape=(), dtype=string) Tensor("ReaderReadV2_1:1", shape=(), dtype=string)


(<tf.Tensor 'batch:0' shape=(128, 84, 84, 3) dtype=float32>,
 <tf.Tensor 'Reshape:0' shape=(128,) dtype=int64>)

In [16]:
import re
import time
from datetime import datetime

TOWER_NAME = 'tower'
NUM_CLASSES = len(label_idx)


def _activation_summary(x):
    """Helper to create summaries for activations.

    Creates a summary that provides a histogram of activations.
    Creates a summary that measures the sparsity of activations.

    Args:
      x: Tensor
    Returns:
      nothing
    """
    # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
    # session. This helps the clarity of presentation on tensorboard.
    tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity',
                      tf.nn.zero_fraction(x))


def _variable_on_cpu(name, shape, initializer):
    """Helper to create a Variable stored on CPU memory.

    Args:
      name: name of the variable
      shape: list of ints
      initializer: initializer for Variable

    Returns:
      Variable Tensor
    """
    with tf.device('/cpu:0'):
        dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
        var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
    return var


def _variable_with_weight_decay(name, shape, stddev, wd):
    """Helper to create an initialized Variable with weight decay.

    Note that the Variable is initialized with a truncated normal distribution.
    A weight decay is added only if one is specified.

    Args:
      name: name of the variable
      shape: list of ints
      stddev: standard deviation of a truncated Gaussian
      wd: add L2Loss weight decay multiplied by this float. If None, weight
          decay is not added for this Variable.

    Returns:
      Variable Tensor
    """
    dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
    var = _variable_on_cpu(
        name,
        shape,
        tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

def inference(images):
    """Build the CIFAR-10 model.

    Args:
      images: Images returned from distorted_inputs() or inputs().

    Returns:
      Logits.
    """
    # We instantiate all variables using tf.get_variable() instead of
    # tf.Variable() in order to share variables across multiple GPU training runs.
    # If we only ran this model on a single GPU, we could simplify this function
    # by replacing all instances of tf.get_variable() with tf.Variable().
    #
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[5, 5, 3, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1)

    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                      name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[5, 5, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                      name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [FLAGS.batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 384],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        _activation_summary(local3)

    # local4
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_weight_decay('weights', shape=[384, 192],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        _activation_summary(local4)
    print('local3', local3)
    print('local4', local4)

    # linear layer(WX + b),
    # We don't apply softmax here because
    # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
    # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [192, NUM_CLASSES],
                                              stddev=1 / 192.0, wd=0.0)
        biases = _variable_on_cpu('biases', [NUM_CLASSES],
                                  tf.constant_initializer(0.0))
        
        print('weights', weights)
        print('biases', biases)
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        
        print('softmax_linear', softmax_linear)
        _activation_summary(softmax_linear)

    return softmax_linear

def _loss(logits, labels):
    """Add L2Loss to all the trainable variables.

    Add summary for "Loss" and "Loss/avg".
    Args:
      logits: Logits from inference().
      labels: Labels from distorted_inputs or inputs(). 1-D tensor
              of shape [batch_size]

    Returns:
      Loss tensor of type float.
    """
    # Calculate the average cross entropy loss across the batch.
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)

    # The total loss is defined as the cross entropy loss plus all of the weight
    # decay terms (L2 loss).
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


def train():
    """Train CIFAR-10 for a number of steps."""
    with tf.Graph().as_default():
        images, labels = get_inputs()
        
        print('labels.get_shape()', labels.get_shape())
        global_step = tf.contrib.framework.get_or_create_global_step()
        
        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = inference(images)
        
        print('images, logits, labels', images, logits, labels)

        # Calculate loss.
        loss = _loss(logits, labels)

        # Build a Graph that trains the model with one batch of examples and
        # updates the model parameters.
        train_op = cifar10.train(loss, global_step)

        class _LoggerHook(tf.train.SessionRunHook):
            """Logs loss and runtime."""

            def begin(self):
                self._step = -1
                self._start_time = time.time()

            def before_run(self, run_context):
                self._step += 1
                return tf.train.SessionRunArgs(loss)  # Asks for loss value.

            def after_run(self, run_context, run_values):
                if self._step % FLAGS.log_frequency == 0:
                    current_time = time.time()
                    duration = current_time - self._start_time
                    self._start_time = current_time

                    loss_value = run_values.results
                    examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
                    sec_per_batch = float(duration / FLAGS.log_frequency)

                    format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                                  'sec/batch)')
                    print(format_str % (datetime.now(), self._step, loss_value,
                                        examples_per_sec, sec_per_batch))

        with tf.train.MonitoredTrainingSession(
                checkpoint_dir=FLAGS.train_dir,
                hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
                       tf.train.NanTensorHook(loss),
                       _LoggerHook()],
                config=tf.ConfigProto(
                    log_device_placement=FLAGS.log_device_placement)) as mon_sess:
            while not mon_sess.should_stop():
                mon_sess.run(train_op)

In [ ]:
import re
import time
from datetime import datetime
import math

TOWER_NAME = 'tower'
NUM_CLASSES = len(label_idx)


def eval_once(saver, summary_writer, top_k_op, summary_op):
    """Run Eval once.

    Args:
      saver: Saver.
      summary_writer: Summary writer.
      top_k_op: Top K op.
      summary_op: Summary op.
    """
    with tf.Session() as sess:
        ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            # Restores from checkpoint
            saver.restore(sess, ckpt.model_checkpoint_path)
            # Assuming model_checkpoint_path looks something like:
            #   /my-favorite-path/cifar10_train/model.ckpt-0,
            # extract global_step from it.
            global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
        else:
            print('No checkpoint file found')
            return

        # Start the queue runners.
        coord = tf.train.Coordinator()
        try:
            threads = []
            for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
                threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                                 start=True))

            num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
            true_count = 0  # Counts the number of correct predictions.
            total_sample_count = num_iter * FLAGS.batch_size
            step = 0
            while step < num_iter and not coord.should_stop():
                predictions = sess.run([top_k_op])
                true_count += np.sum(predictions)
                step += 1

            # Compute precision @ 1.
            precision = true_count / total_sample_count
            print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))

            summary = tf.Summary()
            summary.ParseFromString(sess.run(summary_op))
            summary.value.add(tag='Precision @ 1', simple_value=precision)
            summary_writer.add_summary(summary, global_step)
        except Exception as e:  # pylint: disable=broad-except
            coord.request_stop(e)

        coord.request_stop()
        coord.join(threads, stop_grace_period_secs=10)


def evaluate():
    """Eval CIFAR-10 for a number of steps."""
    with tf.Graph().as_default() as g:
        # Get images and labels for CIFAR-10.
#         eval_data = FLAGS.eval_data == 'test'
#         images, labels = cifar10.inputs(eval_data=eval_data)
        images, labels = get_inputs()

        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = inference(images)
        
        # Calculate predictions.
        top_k_op = tf.nn.in_top_k(logits, labels, 1)

        # Restore the moving average version of the learned variables for eval.
        variable_averages = tf.train.ExponentialMovingAverage(
            cifar10.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        # Build the summary operation based on the TF collection of Summaries.
        summary_op = tf.summary.merge_all()
        summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)
        
        while True:
            eval_once(saver, summary_writer, top_k_op, summary_op)
            if FLAGS.run_once:
                break
            time.sleep(FLAGS.eval_interval_secs)

evaluate()

label_queue <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x119080fd0>
image_file Tensor("ReaderReadV2:1", shape=(), dtype=string)
label Tensor("ReaderReadV2_1:0", shape=(), dtype=string) Tensor("ReaderReadV2_1:1", shape=(), dtype=string)
local3 Tensor("local3/local3:0", shape=(128, 384), dtype=float32)
local4 Tensor("local4/local4:0", shape=(128, 192), dtype=float32)
weights <tf.Variable 'softmax_linear/weights:0' shape=(192, 26) dtype=float32_ref>
biases <tf.Variable 'softmax_linear/biases:0' shape=(26,) dtype=float32_ref>
softmax_linear Tensor("softmax_linear/softmax_linear:0", shape=(128, 26), dtype=float32)
INFO:tensorflow:Restoring parameters from /tmp/kaist_train/model.ckpt-1
2017-09-26 16:00:28.128847: precision @ 1 = 0.041
INFO:tensorflow:Restoring parameters from /tmp/kaist_train/model.ckpt-202
2017-09-26 16:05:43.794269: precision @ 1 = 0.067
INFO:tensorflow:Restoring parameters from /tmp/kaist_train/model.ckpt-202
2017-09-26 16:11:00.054331: precision @ 1 = 0.074